In [1]:
import numpy as np
import gym
import logging
import random

from keras.models import Model
from keras.layers import Dense, Activation, Flatten, Permute
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from gym import error, spaces, utils
from gym.utils import seeding
import math
import keras.backend as K
from keras.layers import Input, Dense, concatenate, Lambda, Conv2D, Reshape
from rl.agents.dqn import DQNAgent
from rl.core import Env
from rl.callbacks import FileLogger, ModelIntervalCheckpoint
#read data as a pandas data frame(ACIM here)
import pandas

import os, sys, csv
import pandas as pd
import numpy as np
import pickle

Using TensorFlow backend.


In [2]:
#找三十個有五年數據的資料
ETF_datapath='/Users/fentingchen/desktop/A3C/ETF/etf_data'
#ETF_datapath='/home/fei-tin/feiting/ETF_5year/etf_data'
data_pathlist = os.listdir(ETF_datapath)[1:]
start_date = '2011-11-16'
data_path = []
i=0
while len(data_path) <30:
    file = ETF_datapath + "/" + data_pathlist[i]
    x = pd.read_csv(file)
    y = x[x['Date'] == start_date]
    if len(y) > 0:
        data_path.append(file)
    i+= 1

In [3]:
#整理data的函式
def drop(data):
    '''
    input: etf dataframe
    output: train dataframe, test dataframe, train date, test date
    '''
    start_date = '2012-02-14'
    start_index = data[data['Date'] == start_date].index
    data_new_reverse = data.iloc[start_index[0]::-1]
    data_close = data_new_reverse["Close"]
    train_df = data_close.reset_index(drop=True)

    return train_df

def data(ETF_datapath):
    test = []
    train = []
    vaild = []
    x= [0]*20 + [1]*5 + [2]*5
    random.shuffle(x)
    for i in range(len(ETF_datapath)):
        data_frame = pandas.read_csv(ETF_datapath[i])
        data = drop(data_frame)
        if x[i] == 0:
                train.append(data.values.reshape(1260,1))
        elif x[i] == 1:
                test.append(data.values.reshape(1260,1))
        elif x[i] == 2:
                vaild.append(data.values.reshape(1260,1))
    return train, test,vaild


In [4]:
#整理資料
data_train,data_test,data_vaild = data(data_path)

In [31]:
data_train[0]

array([[ 30.02    ],
       [ 30.17    ],
       [ 30.450001],
       ..., 
       [ 45.619999],
       [ 45.450001],
       [ 45.779999]])

In [5]:
#模式與訓練次數的選擇
mode = input("Mode?")
step = int(input("Step?"))
number = 0

Mode?test
Step?1


In [6]:
#準備當作輸入的資料
class ETF_Game(Env):
    def __init__(self):
        self.action_space = spaces.Discrete(5)
        self.observation_space = spaces.Box(low = 0, high = math.inf, shape = (20,1))
        self.reward_range = (- math.inf, math.inf)

        #隨便取個名字，方便我們存資料
        self.name = "GameETF"

        #設定隨機的seed
        self.seed()

        #重設遊戲
        self.reset()

    def reset(self):
        self.cycle = 0

        #取得資料作為遊戲盤面
        if mode == 'train':
                i = random.randint(0,19)
                self.board = data_train[i]
        if mode == 'train_final':
                self.board = data_train[number]
        elif mode == 'test':
                self.board = data_test[number]
        #初始化日期(由於需要觀察之前的資料作為data，故從第20天開始)
        self.day = 20

        #最大天數 ＝ 矩陣的最大長度
        self.max_day = self.board.shape[0]

        #初始現金
        self.cash = 20000
        self.min_cash = 20000

        #初始股票
        self.num_of_stock = 0
        self.financial_assets = 0
        self.max_stock = 0
        
        #初始資產價值
        self.assets = self.cash + self.financial_assets
        self.assets_last_time = self.assets

        #初始股票價格 
        self.price = self.board[self.day-1][0]

        #表示遊戲結束與否        
        self.DONE = False

        #回傳初始盤面（gym的規定）
        return self.get_observation()

    def step(self, action):

        #動作會介在0, 1, 2 之間，分別設為買一單位，賣一單位，不動作。(測試)

        # “現金” < "當前股票價格“ 時，不得購買。
        if 10*self.price <= self.cash < 20*self.price and action == 0:
            action = 1
        if self.cash < 10*self.price and (action == 1 or action == 0):
            action = 2

        # “股票數” = 0 時，不得賣出
        if self.num_of_stock == 0 and (action == 3 or action == 4):
            action = 2
        if self.num_of_stock == 1 and action == 4:
            action = 3

        #紀錄
        logging.debug("Action {}".format(action))

        #本次購買量
        purchance_quantity = 0

        # action
        if action == 0:
            purchance_quantity = 20
            logging.debug("Buymore")
        elif action == 1:
            purchance_quantity = 10
            logging.debug("Buy")
        elif action == 2:
            purchance_quantity = 0
            logging.debug("Hold")
        elif action == 3:
            purchance_quantity = -10
            logging.debug("Sell")
        elif action == 4:
            purchance_quantity = -20
            logging.debug("Sellmore")


        #已購買的股票張數更動
        self.num_of_stock += purchance_quantity
        if self.max_stock < self.num_of_stock:
            self.max_stock = self.num_of_stock

        #現存股票價值更動
        self.financial_assets = self.num_of_stock*self.price
        #現金更動
        self.cash -= purchance_quantity*self.price + 0.00145 * abs(purchance_quantity) * self.price
        #交易稅的現今更動，只有賣出的需收取
        if action == 3 or action == 4:
            self.cash -= 0.001*abs(purchance_quantity)*self.price
        
        if self.min_cash < self.cash:
            self.min_cash = self.cash
        #總資產價值更動
        self.assets = self.cash + self.financial_assets

        #reward for this time
        reward = self.assets - self.assets_last_time

        #record today's asset
        self.assets_last_time = self.assets

        #週期數+1
        self.cycle += 1
        self.day += 20

        #update股票價格 
        self.price = self.board[self.day-1][0]

        #get now observation for today
        observation = self.get_observation()

        #當自身資產歸零，或是到達最大天數時，遊戲結束，其餘均繼續進行。
        done = None
        if self.assets == 0 or self.day == self.max_day:
            done = True
            self.DONE = True
        else:
            done = False

        #這裡可以回傳特定資料作為紀錄（格式是字典檔），因為我們目前沒有需要，所以隨便設個空的字典檔。
        info = dict()

        #這裡要回傳什麼，回傳的順序，都是gym規定的

        return observation, reward, done, info



    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_observation(self):
        #get board for last 20 days
        return self.board[self.day-20:self.day]

    #這裡是拿來做test時候的顯示        
    def render(self, mode='human', close=False):
        if close:
            return
        outfile = None
        if self.DONE == True:
            outfile = StringIO() if mode == 'ansi' else sys.stdout
            s = "total assets:" + str(self.assets) + "\n"
            s += "min_cash:" + str(self.min_cash) + "\n"
            #s += "max_num_stock:" + str(self.max_stock) + "\n"
            #s += "cash left" + str(self.cash) + "\n"
            #s += "num of stock left:" + str(self.num_of_stock) + "\n"
            #s += "final price of ETF" + str(self.price) + "\n"
            #s += "financial_assets left" + str(self.financial_assets) + "\n"
            outfile.write(s)
        return outfile

In [7]:
#把我們辛苦架好的遊戲環境作為測試環境
ETF_env = ETF_Game()
nb_actions = ETF_env.action_space.n

#可以執行的動作數
nb_actions
BOARD_INPUT_SHAPE = (20, 1)
WINDOW_LENGTH = 1

#這裡的window_length 是指當我需要傳入包括前幾次畫面作為資料時的東西，
#他是把它當作CNN的channel數一樣的東西
#本來這裡是不需要加的，只是keras-rl寫死了所以我只好傳進去。

#另外，由於資料最後一步是keras-rl處理的，他的順序這樣寫，
#我也只好這樣寫
input_shape = (WINDOW_LENGTH,) + BOARD_INPUT_SHAPE

#設定輸入層的形狀
model_input = Input(shape = input_shape)

#視不同的backend要排一下順序
if K.image_dim_ordering() == 'tf':
    # (width, height, channels)
    permute = Permute((2, 3, 1), input_shape=input_shape)
elif K.image_dim_ordering() == 'th':
    # (channels, width, height)
    permute = Permute((1, 2, 3), input_shape=input_shape)

In [10]:
#把排列的結果套用上去，喬一下我們的原始input
preprocessed_input = permute(model_input)

flat_h_then_v = Flatten()(preprocessed_input)

f = Dense(300 , activation = 'relu')
x = f(flat_h_then_v)

g = Dense(100 , activation = 'relu')
y = g(x)


action = Dense(5)
output_action = action(y)

model = Model(model_input, output_action)

In [9]:
#把排列的結果套用上去，喬一下我們的原始input
preprocessed_input = permute(model_input)

#橫著看～
conv_horizontal_1 =  Conv2D(filters = 32, kernel_size = (1, 6), padding='valid', activation = "relu")

#直的看～
conv_vertical_1 = Conv2D(filters = 32, kernel_size = (5, 1), padding='valid', activation = "relu")

#把處理過的input塞進去
layer_horizontal_1 = conv_horizontal_1(preprocessed_input)
layer_vertical_1 = conv_vertical_1(preprocessed_input)

#再橫著看
conv_horizontal_2 =  Conv2D(filters = 64, kernel_size = (1, 6), padding='valid', activation = "relu")

#在直著看
conv_vertical_2 = Conv2D(filters = 64, kernel_size = (5, 1), padding='valid', activation = "relu")

#交錯塞
layer_h_then_v_2 = conv_vertical_2(layer_horizontal_1)
layer_v_then_h_2 = conv_horizontal_2(layer_vertical_1)

#把上面兩個拉直
flat_h_then_v = Flatten()(layer_h_then_v_2)
flat_v_then_h = Flatten()(layer_v_then_h_2)

#接在一起～


conv_merge = concatenate([flat_h_then_v, flat_v_then_h], name = "Merge_Layer")

f = Dense(300 , activation = 'relu')
x = f(conv_merge)

g = Dense(100 , activation = 'relu')
y = g(x)


action = Dense(5)
output_action = action(y)

model = Model(model_input, output_action)

ValueError: Negative dimension size caused by subtracting 6 from 1 for 'conv2d_1/convolution' (op: 'Conv2D') with input shapes: [?,20,1,1], [1,6,1,32].

In [12]:
preprocessed_input = permute(model_input)
'''
#橫著看～
conv_horizontal_1 =  Conv2D(filters = 32, kernel_size = (3, 3), padding='valid', activation = "relu")

#直的看～
conv_vertical_1 = Conv2D(filters = 32, kernel_size = (3, 3), padding='valid', activation = "relu")

#把處理過的input塞進去
layer_horizontal_1 = conv_horizontal_1(preprocessed_input)
layer_vertical_1 = conv_vertical_1(preprocessed_input)
'''
#再橫著看
conv_horizontal_2 =  Conv2D(filters = 64, kernel_size = (2, 2), padding='valid', activation = "relu")

cnn1 = conv_horizontal_2(preprocessed_input)
#在直著看
conv_vertical_2 = Conv2D(filters = 64, kernel_size = (3, 3), padding='valid', activation = "relu")

cnn2 = conv_vertical_2(cnn1)
conv_merge = Flatten()(cnn2)
'''
#交錯塞
layer_h_then_v_2 = conv_vertical_2(layer_horizontal_1)
layer_v_then_h_2 = conv_horizontal_2(layer_vertical_1)

#把上面兩個拉直
flat_h_then_v = Flatten()(layer_h_then_v_2)
flat_v_then_h = Flatten()(layer_v_then_h_2)

#接在一起～


conv_merge = concatenate([flat_h_then_v, flat_v_then_h], name = "Merge_Layer")
'''
f = Dense(300 , activation = 'relu')
x = f(conv_merge)

g = Dense(100 , activation = 'relu')
y = g(x)


action = Dense(5)
output_action = action(y)

model = Model(model_input, output_action)

#設定記憶體
memory = SequentialMemory(limit=10000, window_length=1)

In [11]:
#設定記憶體
memory = SequentialMemory(limit=10000, window_length=1)

#set policy
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1.,value_min=0, value_test=0, nb_steps=step - 1000000)

#DQN設定
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory = memory, policy = policy,
               nb_steps_warmup=100000, gamma=.95, target_model_update=50,
               train_interval=1, delta_clip=1.)
dqn.compile(Adam(lr=.00001), metrics=['mae'])


In [12]:
if mode == 'train':
    # Okay, now it's time to learn something! We capture the interrupt exception so that training
    # can be prematurely aborted. Notice that you can the built-in Keras callbacks!
    weights_filename = 'dqn_{}_weights.h5f'.format(ETF_env.name)
    checkpoint_weights_filename = 'dqn_' + ETF_env.name + '_weights2_{step}.h5f'
    log_filename = 'dqn_{}_log.json'.format(ETF_env.name)
    callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=100000)]
    callbacks += [FileLogger(log_filename, interval=100)]

    #load weights
    #dqn.load_weights("dqn_Game2048Env_weights2_10000000.h5f")

    dqn.fit(ETF_env, callbacks=callbacks, nb_steps=step, log_interval=10000)

    # After training is done, we save the final weights one more time.
    dqn.save_weights(weights_filename, overwrite=True)

    # Finally, evaluate our algorithm for 10 episodes.
    #dqn.test(env, nb_episodes=10, visualize=False)

elif mode == 'test' or mode =='train_final':
    #weights = "dqn_"+ETF_env.name+"_weights2_10000000.h5f"
    weights = "dqn_ETF_weights2_300_100.h5f"
    print("weights: ", weights)
    if weights:
        weights_filename = weights
    dqn.load_weights(weights_filename)
    if mode == 'test':
        data_len = len(data_test)
    if mode == 'train_final':
        data_len = len(data_train)
    for i in range(data_len):
        number = i
        dqn.test(ETF_env, nb_episodes=1, visualize=True)

weights:  dqn_ETF_weights2_300_100.h5f


ValueError: You are trying to load a weight file containing 7 layers into a model with 3 layers.

In [11]:
for i in range(10):
    if i == 3:
        continue
    print (i)

0
1
2
4
5
6
7
8
9


In [4]:
ETF_datapath='/Users/fentingchen/desktop/A3C/ETF/etf_data'
#ETF_datapath='/home/fei-tin/feiting/ETF_5year/etf_data'
data_pathlist = os.listdir(ETF_datapath)[1:]
data_pathlist[2]="GSD.csv"
start_date = '2011-11-16'
data_path = []
i=0
while len(data_path) < 30:
    try:
        file = ETF_datapath + "/" + data_pathlist[i]
        x = pd.read_csv(file)
        y = x[x['Date'] == start_date]
        if len(y) > 0:
            data_path.append(file)

        i+= 1
    except:
        i+= 1
        continue
print (i)

54


In [5]:
data_path

['/Users/fentingchen/desktop/A3C/ETF/etf_data/AADR.csv',
 '/Users/fentingchen/desktop/A3C/ETF/etf_data/AAXJ.csv',
 '/Users/fentingchen/desktop/A3C/ETF/etf_data/ACWI.csv',
 '/Users/fentingchen/desktop/A3C/ETF/etf_data/ACWV.csv',
 '/Users/fentingchen/desktop/A3C/ETF/etf_data/ACWX.csv',
 '/Users/fentingchen/desktop/A3C/ETF/etf_data/ADRA.csv',
 '/Users/fentingchen/desktop/A3C/ETF/etf_data/ADRD.csv',
 '/Users/fentingchen/desktop/A3C/ETF/etf_data/ADRE.csv',
 '/Users/fentingchen/desktop/A3C/ETF/etf_data/ADRU.csv',
 '/Users/fentingchen/desktop/A3C/ETF/etf_data/ADZ.csv',
 '/Users/fentingchen/desktop/A3C/ETF/etf_data/AFK.csv',
 '/Users/fentingchen/desktop/A3C/ETF/etf_data/AGA.csv',
 '/Users/fentingchen/desktop/A3C/ETF/etf_data/AGF.csv',
 '/Users/fentingchen/desktop/A3C/ETF/etf_data/AGG.csv',
 '/Users/fentingchen/desktop/A3C/ETF/etf_data/AGQ.csv',
 '/Users/fentingchen/desktop/A3C/ETF/etf_data/AGZ.csv',
 '/Users/fentingchen/desktop/A3C/ETF/etf_data/AIA.csv',
 '/Users/fentingchen/desktop/A3C/ETF/et

In [13]:
X_path = '/Users/fentingchen/desktop/A3C/ETF/etf_data/HEFA.csv'
x = pd.read_csv(X_path)

In [14]:
x[::-1]

,b'Date,Open,High,Low,Close,Volume,Adj Close\n2017-02-15,26.85,26.959999,26.85.1,...,24.709999.26,24.709999.27,1700.1,22.408763\n2014-02-14,24.52.15,24.52.16,24.52.17,24.52.18,1300.2,22.236459\n'


In [16]:
ETF_datapath='/Users/fentingchen/desktop/A3C/ETF/etf_data'
data_pathlist = os.listdir(ETF_datapath)[1:]
start_date = '2011-11-16'
data_path = []
data_name = []
i=0

while len(data_path) <30:
    try:
        file = ETF_datapath + "/" + data_pathlist[i]
        x = pd.read_csv(file)
        y = x[x['Date'] == start_date]
        if len(y) > 0:
            data_path.append(file)
            data_name.append(data_pathlist[i])
        i+= 1
    except:
        i+= 1
        continue
